In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
import os
os.chdir('C:\\code\\activ')

In [3]:
df_name = 'nov_nine_var.xlsx'

In [4]:
df = pd.read_excel('./documents/'+df_name).iloc[:,1::]
X = df.iloc[:,0:21]
y = df.iloc[:,21::].to_numpy()

In [5]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X.shape

(431, 21)

In [6]:
dnn_model = tf.keras.models.load_model('./model/dnn.h5')

In [7]:
def my_loss(real, X_pred):
    y_pred = dnn_model(X_pred)
    return tf.math.subtract(y_pred, 0.50384917)

In [8]:
generator = tf.keras.Sequential()
generator.add(tf.keras.layers.Dense(128, input_dim=21, activation='relu'))
generator.add(tf.keras.layers.Dense(64, activation='relu'))
generator.add(tf.keras.layers.Dense(32, activation='relu'))
generator.add(tf.keras.layers.Dense(16, activation='relu'))
generator.add(tf.keras.layers.Dense(8, activation='relu'))
generator.add(tf.keras.layers.Dense(4, activation='relu'))
generator.add(tf.keras.layers.Dense(21, activation='sigmoid'))

generator.compile(loss=my_loss, optimizer='adam')

In [9]:
result = []
pop_result = []
for i in range(1000):
    noise = tf.random.normal([1, 21])
    X_pred = generator.predict(noise, verbose=0)
    loss = my_loss(noise, X_pred)

    pop = dnn_model.predict(X_pred, verbose=0)
    generator.train_on_batch(noise, X_pred)
    
    result.append(X_pred)
    pop_result.append(pop)

    print(f"epoch: {i}, loss: {loss}, pop: {pop}")

epoch: 0, loss: [[-0.0281094]], pop: [[0.47573975]]
epoch: 1, loss: [[-0.02794418]], pop: [[0.47590497]]
epoch: 2, loss: [[-0.02802292]], pop: [[0.47582623]]
epoch: 3, loss: [[-0.02891517]], pop: [[0.47493398]]
epoch: 4, loss: [[-0.02907971]], pop: [[0.47476944]]
epoch: 5, loss: [[-0.02917266]], pop: [[0.4746765]]
epoch: 6, loss: [[-0.02924424]], pop: [[0.4746049]]
epoch: 7, loss: [[-0.02941889]], pop: [[0.47443026]]
epoch: 8, loss: [[-0.02930641]], pop: [[0.47454274]]
epoch: 9, loss: [[-0.02965969]], pop: [[0.47418946]]
epoch: 10, loss: [[-0.02955464]], pop: [[0.4742945]]
epoch: 11, loss: [[-0.02996141]], pop: [[0.47388774]]
epoch: 12, loss: [[-0.03004658]], pop: [[0.47380257]]
epoch: 13, loss: [[-0.02931914]], pop: [[0.47453]]
epoch: 14, loss: [[-0.02967241]], pop: [[0.47417673]]
epoch: 15, loss: [[-0.02974564]], pop: [[0.4741035]]
epoch: 16, loss: [[-0.03036821]], pop: [[0.47348094]]
epoch: 17, loss: [[-0.03006181]], pop: [[0.47378734]]
epoch: 18, loss: [[-0.03019831]], pop: [[0.473

In [10]:
result = np.array(result).reshape(1000, 21)
X_result = scaler.inverse_transform(result)
pop_result = np.array(pop_result).reshape(1000, 1)

In [11]:
print(pop_result.max())
print(pop_result.min())
print(pop_result.mean())

0.47590497
0.3006039
0.31495723


In [12]:
df = np.append(X_result, pop_result, axis=1)

In [13]:
pd_df = pd.DataFrame(df)
pd_df.to_excel('./documents/result/basic.xlsx')